### Advent of Code 2022
#### Day 5: Supply Stacks

You're operating a crane, following a set of instructions telling you how many boxes to move from one stack to another.  After you follow the instructions, the top boxes in the stacks spell out a message.

**Part 1.** What's the message if the crane moves one box at a time?

**Part 2.** What's the message if the crane moves all of the boxes at once?

I'm bringing in my usual utility functions.

In [1]:
(require racket
         advent-of-code
         threading
         (only-in relation ->string ->list ->number)
         (only-in algorithms chunks-of))


The data file in this one is unusual: the first 10 lines are a pictorial representation of the stacks of boxes, and the remaining lines are instructions in the form `move X from Y to Z`.

Tackling the boxes first, I can get the contents of each stack if I treat the picture as an array and transpose it, drop the first row that contains only brackets, then only take rows 1, 5, 9..., which are the rows with letters in them and trim the leading spaces from each row.  I then use this list to create a hashtable.

In [ ]:
(define assignments (~> (fetch-aoc-input (find-session) 2022 5) (string-split "\n")))

(define crates-list
  (~>> assignments
       (take _ 8)
       (map (λ~>> ->list))
       (apply map list _)
       rest
       (chunks-of _ 4)
       (map (λ~> first ->string string-trim ->list) _)))

(define crates
  (for/hash ([c (in-list crates-list)] [i (in-naturals 1)])
    (values i c)))


The instructions are a little easier; this is just a regex and destructuring like in Day 4.

In [ ]:
(struct instruction (n from to))

(define (parse-instruction str)
  (match str
    [(regexp #px"move (\\d+) from (\\d) to (\\d)" (list _ n from to))
     (instruction (->number n) (->number from) (->number to))]))

(define instructions (~>> assignments (drop _ 10) (map parse-instruction)))


##### Part 1

The main function to iterate over the list of instructions is the same for both parts, except for whether the boxes taken off of the origin stack are reversed or not when they end up on the destination stack.  They end up reversed if they're taken off one at a time, and don't reverse if the whole stack is picked up at once.

Once I've iterated through all the instructions, the `#:result` clause parses the final crate state.

In [ ]:
(define (find-crate-message cs [reverse-function reverse])
  (for/fold ([current-crates cs]
             #:result (~>> (hash-values current-crates) (map first) (apply string)))
            ([i (in-list instructions)])
    (match-define (instruction n from to) i)
    (define taken (~> (hash-ref current-crates from) (take _ n) reverse-function))
    (~> current-crates
        (hash-update _ from (λ (v) (drop v n)))
        (hash-update _ to (λ (v) (append taken v))))))

(find-crate-message crates)

##### Part 2

The result, if the moved boxes don't get flipped:

In [ ]:
(find-crate-message crates identity)